In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statistics
%matplotlib inline
from datetime import datetime
from pandas import Series
import warnings
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

# Data Pre-processing
### Import Time Series Data

- read the footfall data
- convert the 'timestamp' colum to pandas timestamp data type
- resample the time unit to one hour and interpolate the missing data using linear method.
- extract features from timestamp
- export as csv

In [8]:
def preprocess(year): 
    """
        read the footfall data and resample the time to one hour unit.
        
    """    
    for month in range(12):
        if month < 9 :
            FFraw = pd.read_csv(year + '/' + year + '-0' + str(month + 1) + '.csv')
            
            FFraw['timestamp'] = FFraw['timestamp'].str.replace('T', ' ')
            FFraw['timestamp'] = FFraw['timestamp'].str.replace('Z', '')
            FFraw.timestamp = pd.to_datetime(FFraw.timestamp, format = '%Y-%m-%d %H:%M:%S')

            FFraw.index = FFraw.timestamp
            FFraw_h = FFraw.groupby('location').resample('H').mean()
            FFraw_h = FFraw_h.reset_index(level = ['timestamp'])
            FFraw_h.timestamp = pd.to_datetime(FFraw_h.timestamp, format = '%Y-%m-%d %H:%M:%S')
            FFraw_h.footfall = FFraw_h.footfall * 12
            FFraw_h['footfall'].replace(0, np.nan, inplace = True)
            FFraw_h['footfall'] = FFraw_h['footfall'].interpolate(method='linear')          

            FFraw_h['year'] = FFraw_h.timestamp.dt.year
            FFraw_h['month'] = FFraw_h.timestamp.dt.month
            FFraw_h['day'] = FFraw_h.timestamp.dt.day
            FFraw_h['hour'] = FFraw_h.timestamp.dt.hour
            FFraw_h['day_of_week'] = FFraw_h.timestamp.dt.dayofweek + 1

            FF_preprocess = FFraw_h.reset_index(drop = True)
            FF_preprocess.to_csv('preprocess/'+ year + '-0' + str(month + 1) + '.csv')

        else:
            FFraw = pd.read_csv(year + '/' + year + '-' + str(month + 1) + '.csv')
            
            FFraw['timestamp'] = FFraw['timestamp'].str.replace('T', ' ')
            FFraw['timestamp'] = FFraw['timestamp'].str.replace('Z', '')
            FFraw.timestamp = pd.to_datetime(FFraw.timestamp, format = '%Y-%m-%d %H:%M:%S')

            FFraw.index = FFraw.timestamp
            FFraw_h = FFraw.groupby('location').resample('H').mean()
            FFraw_h = FFraw_h.reset_index(level = ['timestamp'])
            FFraw_h.timestamp = pd.to_datetime(FFraw_h.timestamp, format = '%Y-%m-%d %H:%M:%S')
            FFraw_h.footfall = FFraw_h.footfall * 12
            FFraw_h['footfall'].replace(0, np.nan, inplace = True)
            FFraw_h['footfall'] = FFraw_h['footfall'].interpolate(method='linear')          

            FFraw_h['year'] = FFraw_h.timestamp.dt.year
            FFraw_h['month'] = FFraw_h.timestamp.dt.month
            FFraw_h['day'] = FFraw_h.timestamp.dt.day
            FFraw_h['hour'] = FFraw_h.timestamp.dt.hour
            FFraw_h['day_of_week'] = FFraw_h.timestamp.dt.dayofweek + 1

            FF_preprocess = FFraw_h.reset_index(drop = True)
            FF_preprocess.to_csv('preprocess/'+ year + '-' + str(month + 1) + '.csv')


In [10]:
preprocess('2018')
preprocess('2019')
preprocess('2020')

### Stack FF Data with Same Year

In [13]:
def stack_footfall(year):
    footfall = pd.read_csv('preprocess/'+ year +'-01.csv', index_col = 0)
    
    for i in range(11):
        if i < 8:
            footfall = footfall.append(
                pd.read_csv('preprocess/'+ year +'-0' + str(i + 2) + '.csv', index_col = 0))
        else:
            footfall = footfall.append(
                pd.read_csv('preprocess/'+ year +'-' + str(i + 2) + '.csv', index_col = 0)) 
            
    footfall.timestamp = pd.to_datetime(footfall.timestamp, format = '%Y-%m-%d %H:%M:%S') 
    
    return footfall

In [14]:
ff2018 = stack_footfall('2018')
ff2019 = stack_footfall('2019')
ff2020 = stack_footfall('2020')

### Stack FF Data of 3 Years

In [15]:
ff = ff2018.append(ff2019).append(ff2020)
ff.to_csv('stackfootfall.csv')

### Load FF Data

In [ ]:
ff = pd.read_csv('stackfootfall.csv')
ff.timestamp = pd.to_datetime(footfall.timestamp, format = '%Y-%m-%d %H:%M:%S') 

### Exclude Locations with Too Much Missing Data

identify 